In [ ]:

import re
import sys
import os
import random
import string
import time
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras.layers import add
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, TimeDistributed
from tensorflow.keras.layers import Embedding, Dropout, Bidirectional, Concatenate, Lambda
import glob
import pandas as pd
np.random.seed(1)


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

os.chdir("/content/drive/Othercomputers/Dell")

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/Othercomputers/Dell")
!pwd

/content/drive/Othercomputers/Dell


In [ ]:
path = !pwd
path
#path = /content/drive/Othercomputers/Dell/Action-Segmentation-Project

['/content/drive/Othercomputers/Dell']

In [ ]:
#get all the optofoce_data.csv

# files = ['/media/sf_Action-Segmentation-Project/avocado','']
# for file in files:
#     for name in glob.glob(f"{file}/clutter/[0-9]*/optoforce_data.csv"):
#         print(name)

# files = glob.glob(f"/media/sf_Action-Segmentation-Project/avocado/clutter/[0-9]*/optoforce_data.csv")
files = glob.glob(f"/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/[0-9]*/optoforce_data.csv")
files

['/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/7/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/4/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/5/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/9/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/8/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/2/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/3/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/10/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/6/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Projec

In [ ]:
#labels = glob.glob(f"/media/sf_Action-Segmentation-Project/avocado/clutter/[0-9]*/labels")
labels = glob.glob(f"/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/[0-9]*/labels")

labels

['/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/7/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/4/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/5/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/9/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/8/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/2/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/3/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/10/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/6/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/1/labels']

In [ ]:
frames = []
data = []
for f in files:
    data_df = pd.read_csv(f)
    data_df["label"] = ""
    frames.append(data_df)



In [ ]:
len(frames)

10

In [ ]:
frames[0]

,time,index_x,index_y,index_z,middle_x,middle_y,middle_z,ring_x,ring_y,ring_z,thumb_x,thumb_y,thumb_z,label
0,1635185477459841536,-0.16,-0.16,0.64,-0.16,-0.40,0.08,-0.08,-0.16,0.32,0.32,0.00,1.12,
1,1635185477460862720,-0.16,-0.16,0.64,-0.24,-0.40,0.08,-0.16,-0.16,0.32,0.32,0.00,1.20,
2,1635185477461578496,-0.16,-0.16,0.64,-0.08,-0.32,0.08,-0.16,-0.08,0.32,0.24,-0.08,1.20,
3,1635185477462545664,-0.08,-0.16,0.64,-0.08,-0.24,0.00,-0.16,-0.08,0.32,0.24,0.00,1.20,
4,1635185477463597312,-0.08,-0.08,0.64,-0.08,-0.24,0.00,-0.16,0.00,0.32,0.24,0.00,1.20,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72505,1635185564093448960,0.08,-0.56,0.08,0.40,-0.16,-0.08,0.16,0.00,-0.40,0.56,-0.08,0.40,
72506,1635185564094279424,0.08,-0.64,0.08,0.40,-0.24,-0.08,0.16,0.00,-0.40,0.48,-0.16,0.40,
72507,1635185564095298560,0.00,-0.64,0.00,0.32,-0.24,-0.08,0.00,-0.08,-0.32,0.48,-0.16,0.40,
72508,1635185564096268032,-0.08,-0.64,0.00,0.32,-0.24,-0.08,0.00,-0.08,-0.40,0.40,-0.24,0.40,


In [ ]:
action_segment_td = []
ground_truth_actions = []

for l in labels:
    td_per_file = []
    gt_actions_per_file = []
    with open (l) as f:
        for line in f:
            x, y  = line.split(';')
            #start_time, end_time = x.split(':')
            td_per_file.append(x)
            gt_actions_per_file.append(y.strip('\n'))
        action_segment_td.append(td_per_file)
        ground_truth_actions.append(gt_actions_per_file)


In [ ]:
action_segment_td #timestmps

[['1635185484881988107:1635185508960098923',
  '1635185508960098923:1635185540489498512',
  '1635185540489498512:1635185544057936064',
  '1635185544057936064:1635185549350041664',
  '1635185549350041664:1635185554816474229',
  '1635185554816474229:1635185560655390372'],
 ['1634917079582315741:1634917108946914859',
  '1634917108946914859:1634917129420464685',
  '1634917129420464685:1634917131539426547',
  '1634917131539426547:1634917134957114204',
  '1634917134957114204:1634917140855676126',
  '1634917140855676126:1634917153391803434'],
 ['1634918557303008582:1634918568217991596',
  '1634918568217991596:1634918593817386818',
  '1634918593817386818:1634918646595079287',
  '1634918646595079287:1634918651382668779',
  '1634918651382668779:1634918657623002640',
  '1634918657623002640:1634918664418463962'],
 ['1635186448159756509:1635186459390779330',
  '1635186459390779330:1635186464698664958',
  '1635186464698664958:1635186467998475334',
  '1635186467998475334:1635186475265396949',
  '1635

In [ ]:
len(ground_truth_actions) #labels


10

In [ ]:
ground_truth_actions


[['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in', 'manipulate', 'move-in', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'move-out',
  'move-in',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'move-out',
  'move-in',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'grasp',
  'pick-up',
  'move-out',
  'drop']]

In [ ]:
for df, duration_of_actions, labels in zip(frames,action_segment_td, ground_truth_actions):
    condition = []
    for actions in duration_of_actions:
        start_time, end_time = actions.split(':')
        condition.append(df['time'].between(int(start_time),int(end_time)))

    df['label'] = np.select(condition,labels,default=None)
    df.dropna(inplace=True)


In [ ]:
len(frames[0])


62956

In [ ]:
unique_actions = set()

for frame in frames:
    for label in frame['label']:
        unique_actions.add(label)


In [ ]:
unique_actions


{'drop', 'grasp', 'manipulate', 'move-in', 'move-out', 'pick-up'}

In [ ]:
one_hot_encoding_acts = pd.get_dummies(list(unique_actions))
one_hot_encoding_acts


,drop,grasp,manipulate,move-in,move-out,pick-up
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,1,0
5,1,0,0,0,0,0


In [ ]:
actions_per_seq = []
for frame in frames:
    action_encodings = []
    for i in range(0, len(frame)):
        action_encodings.append(one_hot_encoding_acts[frame['label'].iloc[i]])
    actions_per_seq.append(action_encodings)

In [ ]:
len(actions_per_seq)


10

In [ ]:
len(actions_per_seq[0])


62956

In [ ]:
frames[0].columns

Index(['time', 'index_x', 'index_y', 'index_z', 'middle_x', 'middle_y',
       'middle_z', 'ring_x', 'ring_y', 'ring_z', 'thumb_x', 'thumb_y',
       'thumb_z', 'label'],
      dtype='object')

In [ ]:
features = ['index_x', 'index_y', 'index_z','middle_x','middle_y','middle_z','ring_x','ring_y','ring_z','thumb_x', 'thumb_y', 'thumb_z' ]
numeric_features = frames[0][features]
#numeric_features.head()
np.array(numeric_features)

array([[ 0.16, -0.24,  0.48, ...,  0.4 ,  0.08,  1.2 ],
       [ 0.  , -0.24,  0.48, ...,  0.4 ,  0.  ,  1.2 ],
       [ 0.  , -0.16,  0.48, ...,  0.24,  0.  ,  1.2 ],
       ...,
       [ 0.  , -0.56,  0.16, ...,  0.8 , -0.56,  0.64],
       [ 0.08, -0.56,  0.24, ...,  0.8 , -0.56,  0.64],
       [ 0.16, -0.48,  0.24, ...,  0.8 , -0.4 ,  0.64]])

In [ ]:
numeric_features.head()


,index_x,index_y,index_z,middle_x,middle_y,middle_z,ring_x,ring_y,ring_z,thumb_x,thumb_y,thumb_z
6462,0.16,-0.24,0.48,0.16,-0.24,0.08,0.08,0.00,-0.16,0.40,0.08,1.20
6463,0.00,-0.24,0.48,0.08,-0.24,0.00,0.00,0.00,-0.16,0.40,0.00,1.20
6464,0.00,-0.16,0.48,0.00,-0.32,0.00,-0.16,-0.08,-0.16,0.24,0.00,1.20
6465,0.00,-0.16,0.48,-0.08,-0.32,0.00,-0.16,-0.16,-0.16,0.24,-0.08,1.12
6466,0.00,-0.24,0.40,-0.08,-0.32,0.08,-0.16,-0.16,-0.16,0.24,-0.08,1.12


In [ ]:
numeric_features_per_seq = [np.array(frames[i][features]) for i in range(len(frames))]

In [ ]:
labels_per_seq =   [np.array(actions_per_seq[i]) for i in range(len(actions_per_seq))]

In [ ]:
labels_per_seq[0].shape[0]

62956

In [ ]:
longest_seq = 0
for seq in numeric_features_per_seq:
  print(seq.shape[0])

  if seq.shape[0] >= longest_seq:
    longest_seq = seq.shape[0]

longest_seq


62956
62000
86756
31740
115690
47585
106329
65967
118387
101627


118387

In [ ]:
def CausalWindowGenerator(sequence, labels, window_length=3, stride =1, n_th_frame = 1):
  """
  squence = todo
  actions = action per time step
  window - xt-d - xt
  """
  sequence_len = len(sequence)
  for t in range(0,sequence_len,stride):
    if t >= window_length-1:
      start_index = t-window_length+1
      x = sequence[start_index:t+1:n_th_frame,:]
      y = labels[start_index:t+1:n_th_frame,:]
    else:
      padding_x = np.tile(sequence[0], ((window_length-t-1),1))
      x = sequence[0:t+1:n_th_frame, :]
      x = np.concatenate((padding_x,x),axis = 0)
      
      padding_y =  np.tile(labels[0], ((window_length-t-1),1))
      y = labels[0:t+1:n_th_frame, :]
      y = np.concatenate((padding_y,y),axis = 0)
  
    yield x, y

In [ ]:
test_for_single_seq = np.random.rand(3,5,12)
test_for_single_seq

array([[[0.40670287, 0.35641009, 0.15115604, 0.85647306, 0.10200765,
         0.95312101, 0.16681381, 0.43940444, 0.16041639, 0.87927176,
         0.47845035, 0.3991534 ],
        [0.27547018, 0.05164124, 0.13871462, 0.7786384 , 0.17805335,
         0.58343853, 0.92812382, 0.10766388, 0.45264432, 0.1559925 ,
         0.30421083, 0.76412394],
        [0.62692194, 0.71109532, 0.19346057, 0.06843106, 0.879656  ,
         0.31984892, 0.60448001, 0.95521018, 0.41579212, 0.19589157,
         0.16602848, 0.62480708],
        [0.05629367, 0.32855282, 0.98041183, 0.77159738, 0.20933243,
         0.58528221, 0.43978807, 0.61355417, 0.31997143, 0.11189976,
         0.31301464, 0.99393789],
        [0.16577418, 0.46292009, 0.63551716, 0.03708797, 0.25327427,
         0.87021266, 0.13963125, 0.01807095, 0.68971395, 0.87107777,
         0.35550828, 0.39679911]],

       [[0.86291172, 0.93225717, 0.4507589 , 0.97499131, 0.69793712,
         0.75361048, 0.57777436, 0.13460615, 0.80769903, 0.10466861,


In [ ]:
test_labels_for_single_seq = labels_per_seq[0][np.random.choice(np.arange(labels_per_seq[0].shape[0]), size = (3,5,))]
test_labels_for_single_seq

array([[[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]],

       [[1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0]]], dtype=uint8)

In [ ]:
# testing single sequences
# for seq, l  in CausalWindowGenerator(test_for_single_seq,test_labels_for_single_seq):
#   print(seq)

#testing sequences in each batch
X_train = []
for x,y in zip(test_for_single_seq,test_labels_for_single_seq):
    temp = []
    for a, b in CausalWindowGenerator(x,y):
      print(a)
      temp.append(a)
      #break
    X_train.append(temp)

    #break
    print("----------")
    #X_train.append(temp))
    
#X_train

[[0.40670287 0.35641009 0.15115604 0.85647306 0.10200765 0.95312101
  0.16681381 0.43940444 0.16041639 0.87927176 0.47845035 0.3991534 ]
 [0.40670287 0.35641009 0.15115604 0.85647306 0.10200765 0.95312101
  0.16681381 0.43940444 0.16041639 0.87927176 0.47845035 0.3991534 ]
 [0.40670287 0.35641009 0.15115604 0.85647306 0.10200765 0.95312101
  0.16681381 0.43940444 0.16041639 0.87927176 0.47845035 0.3991534 ]]
[[0.40670287 0.35641009 0.15115604 0.85647306 0.10200765 0.95312101
  0.16681381 0.43940444 0.16041639 0.87927176 0.47845035 0.3991534 ]
 [0.40670287 0.35641009 0.15115604 0.85647306 0.10200765 0.95312101
  0.16681381 0.43940444 0.16041639 0.87927176 0.47845035 0.3991534 ]
 [0.27547018 0.05164124 0.13871462 0.7786384  0.17805335 0.58343853
  0.92812382 0.10766388 0.45264432 0.1559925  0.30421083 0.76412394]]
[[0.40670287 0.35641009 0.15115604 0.85647306 0.10200765 0.95312101
  0.16681381 0.43940444 0.16041639 0.87927176 0.47845035 0.3991534 ]
 [0.27547018 0.05164124 0.13871462 0.77

In [ ]:
np.array(X_train).shape #batch_size,number_of_windows,window_size,features

(3, 5, 3, 12)

In [ ]:
X_trian = np.array(X_train)

In [ ]:
X_train_reshaped = np.reshape(X_train,(15,3,12))

In [ ]:
X_train_reshaped

array([[[0.40670287, 0.35641009, 0.15115604, 0.85647306, 0.10200765,
         0.95312101, 0.16681381, 0.43940444, 0.16041639, 0.87927176,
         0.47845035, 0.3991534 ],
        [0.40670287, 0.35641009, 0.15115604, 0.85647306, 0.10200765,
         0.95312101, 0.16681381, 0.43940444, 0.16041639, 0.87927176,
         0.47845035, 0.3991534 ],
        [0.40670287, 0.35641009, 0.15115604, 0.85647306, 0.10200765,
         0.95312101, 0.16681381, 0.43940444, 0.16041639, 0.87927176,
         0.47845035, 0.3991534 ]],

       [[0.40670287, 0.35641009, 0.15115604, 0.85647306, 0.10200765,
         0.95312101, 0.16681381, 0.43940444, 0.16041639, 0.87927176,
         0.47845035, 0.3991534 ],
        [0.40670287, 0.35641009, 0.15115604, 0.85647306, 0.10200765,
         0.95312101, 0.16681381, 0.43940444, 0.16041639, 0.87927176,
         0.47845035, 0.3991534 ],
        [0.27547018, 0.05164124, 0.13871462, 0.7786384 , 0.17805335,
         0.58343853, 0.92812382, 0.10766388, 0.45264432, 0.1559925 ,
